# Práctica 1: Resolución de problemas de clasificación y análisis experimental

### Pilar Navarro Ramírez




In [3]:
import pandas as pd

Leemos el fichero con los datos que vamos a estudiar, indicado que los valores con '?' son valores perdidos

In [4]:
raw_data=pd.read_csv("mamografias.csv",na_values=["?"])

In [5]:
raw_data.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,L,5.0,3.0,maligno
1,4.0,43.0,R,1.0,NaN,maligno
2,5.0,58.0,I,5.0,3.0,maligno
3,4.0,28.0,R,1.0,3.0,benigno
4,5.0,74.0,R,5.0,NaN,maligno


In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   BI-RADS   959 non-null    float64
 1   Age       956 non-null    float64
 2   Shape     961 non-null    object 
 3   Margin    913 non-null    float64
 4   Density   885 non-null    float64
 5   Severity  961 non-null    object 
dtypes: float64(4), object(2)
memory usage: 45.2+ KB


In [7]:
raw_data.shape

(961, 6)

In [8]:
raw_data.describe()

,BI-RADS,Age,Margin,Density
count,959.000000,956.000000,913.000000,885.000000
mean,4.296142,55.487448,2.796276,2.910734
std,0.706291,14.480131,1.566546,0.380444
min,0.000000,18.000000,1.000000,1.000000
25%,4.000000,45.000000,1.000000,3.000000
50%,4.000000,57.000000,3.000000,3.000000
75%,5.000000,66.000000,4.000000,3.000000
max,6.000000,96.000000,5.000000,4.000000


In [9]:
raw_data.columns = [col.lower() for col in raw_data]

In [10]:
raw_data['shape'].describe()

count     961
unique      5
top         I
freq      400
Name: shape, dtype: object

In [11]:
raw_data['severity'].describe()

count         961
unique          2
top       benigno
freq          516
Name: severity, dtype: object

## 1. Preprocesamiento de los Datos

Transformamos los atributos de tipo cadena en valores numéricos:  

In [12]:
from sklearn import preprocessing

le=preprocessing.LabelEncoder()

numeric_data=raw_data.copy()

numeric_data['shape']=le.fit_transform(raw_data['shape'])
numeric_data['severity']=le.fit_transform(raw_data['severity'])


In [116]:
numeric_data.head()

,bi-rads,age,shape,margin,density,severity
0,5.0,67.0,1,5.0,3.0,1
1,4.0,43.0,4,1.0,NaN,1
2,5.0,58.0,0,5.0,3.0,1
3,4.0,28.0,4,1.0,3.0,0
4,5.0,74.0,4,5.0,NaN,1


In [14]:
numeric_data.describe()

,bi-rads,age,shape,margin,density,severity
count,959.000000,956.000000,961.000000,913.000000,885.000000,961.000000
mean,4.296142,55.487448,1.754422,2.796276,2.910734,0.463059
std,0.706291,14.480131,1.690674,1.566546,0.380444,0.498893
min,0.000000,18.000000,0.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,0.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,1.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,3.000000,4.000000,3.000000,1.000000
max,6.000000,96.000000,4.000000,5.000000,4.000000,1.000000


### Valores perdidos

Tratamos ahora con los valores perdidos. Para ello probamos un par opciones que se indican a continuación.

**Borrado de columnas**

Empezamos probando la opción más fácil que es eliminar las filas que tengan valores perdidos.

In [15]:
less_data=numeric_data.copy()
less_data=less_data.dropna()

In [16]:
less_data[(less_data['severity']==0)].count()

bi-rads     437
age         437
shape       437
margin      437
density     437
severity    437
dtype: int64

In [17]:
less_data.count()

bi-rads     847
age         847
shape       847
margin      847
density     847
severity    847
dtype: int64

Perdemos algunos datos de entrenamiento, pero nos quitamos de problemas con los valores perdidos y la diferencia de los valores estadísticos es muy pequeña. Sólo cambia ligeramente la media y la desviación típica. 

**Reemplazar valores perdidos por el valor más frecuente**

Sustituimos ahora los valores perdidos por el valor más fecuente en la columna afectada.

In [18]:
from sklearn import impute

In [19]:
imputer=impute.SimpleImputer(strategy="most_frequent")

In [20]:
replaced_data=imputer.fit_transform(numeric_data)

In [21]:
replaced_data=pd.DataFrame(replaced_data)
replaced_data.columns=numeric_data.columns

replaced_data.describe()

,bi-rads,age,shape,margin,density,severity
count,961.000000,961.000000,961.000000,961.000000,961.000000,961.000000
mean,4.295525,55.505723,1.754422,2.706556,2.917794,0.463059
std,0.705684,14.444586,1.690674,1.576272,0.365869,0.498893
min,0.000000,18.000000,0.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,0.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,1.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,3.000000,4.000000,3.000000,1.000000
max,6.000000,96.000000,4.000000,5.000000,4.000000,1.000000


### Variables correladas

Estudiamos ahora la correlación entre los diferentes atributos de los que disponemos.

In [86]:
numeric_data.corr(method='spearman')

,bi-rads,age,shape,margin,density,severity
bi-rads,1.000000,0.354125,-0.492195,0.520715,0.090118,0.616910
age,0.354125,1.000000,-0.351511,0.385146,0.047433,0.428576
shape,-0.492195,-0.351511,1.000000,-0.711305,-0.071306,-0.555479
margin,0.520715,0.385146,-0.711305,1.000000,0.111100,0.566320
density,0.090118,0.047433,-0.071306,0.111100,1.000000,0.075020
severity,0.616910,0.428576,-0.555479,0.566320,0.075020,1.000000


Podemos observar que los atributos *shape* y *margin* están correlacionados, pues el valor del coeficiente de correlación de Spearman es -0.711, lo cual está próximo a -1.

Así pues, probamos a eliminar uno de estos atributos, por ejemplo *margin*.

In [24]:
cols = [col for col in numeric_data.columns if col not in ['margin']]    
corr_data=numeric_data[cols]
corr_data=corr_data.dropna()
corr_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878 entries, 0 to 960
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   bi-rads   878 non-null    float64
 1   age       878 non-null    float64
 2   shape     878 non-null    int64  
 3   density   878 non-null    float64
 4   severity  878 non-null    int64  
dtypes: float64(3), int64(2)
memory usage: 41.2 KB


### Elegir atributo a precedir

Finalmente, para cada DataFrame resultante del preprocesado, vamos a seleccionar el atributo que queremos predecir(target) y separarlo del resto de atributos, los cuales usaremos para predecir dicho atributo (en nuestro caso sería 'severity')

In [26]:
def splitData(data):
    cols = [col for col in data.columns if col not in ['severity']]    
    return data[cols], data['severity']
   
less_attributes,less_target=splitData(less_data)
replaced_attributes,replaced_target =splitData(replaced_data)
corr_attributes,corr_target=splitData(corr_data)

## 2. Configuración de algoritmos

En todos los casos usaremos la validación cruzada con k=5 para dividir el conjunto de datos en el conjunto de entrenamiendo y el conjunto de test. Para ello, recurrimos a la función *Kfold* de sklearn.

In [27]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt

In [28]:
'''
Función genérica para resolver el problema de clasificación.
Muestra por pantalla los resultados de las métricas de 
evaluación tras entrenar el modelo y predecir las clases sobre el 
conjunto de datos de test
    
    X -> Conjunto de atributos que se usarán para predecir la clases
    Y -> Etiquetas de las clases
    clf -> Clasificador que se usará para resolver el problema 
    
    No devuelve nada 
   
'''

def clasificacion(X,Y,clf):
    kf=KFold(n_splits=5,shuffle=True,random_state=100)

    f1,auc,accuracy=0,0,0
    cm=np.zeros((2,2))

    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

        clf.fit(x_train, y_train)
        pred=clf.predict(x_test)

        auc=auc+roc_auc_score(y_test,pred)

        accuracy=accuracy+accuracy_score(y_test, pred, normalize=True)
        f1=f1+f1_score(y_test, pred)
        cm=cm+confusion_matrix(y_test,pred,normalize='true')

    print("Precisión: ", accuracy/5)
    print("AUC: ", auc/5)
    print("F1-score: ",f1/5)
    print("Confusion Matrix:\n ", cm/5)

### Linear SVC

In [29]:
from sklearn.svm import LinearSVC

def svm(X,Y,C=1.0,class_weight=None,max_iter=1000):
    clf=LinearSVC(C=C,class_weight=class_weight,random_state=10,max_iter=max_iter)
    clasificacion(X,Y,clf)

Determinar cuales de los datos preprocesados es mejor.

In [479]:
print("Linear Support Vector Classification")

print("Datos con instancias eliminadas:\n")
svm(less_attributes,less_target,1.0,None,1000000)
print("\n")

print("Datos con valores perdidos reemplazados por la moda:\n")
svm(replaced_attributes,replaced_target,1.0,None,1000000)
print("\n")

print("Datos con atributos correlados eliminados:\n")
svm(corr_attributes,corr_target,1.0,None,1000000)

Linear Support Vector Classification
Datos con instancias eliminadas:

Precisión:  0.825255830142708
AUC:  0.8259227956975396
F1-score:  0.8227365658017838
Confusion Matrix:
  [[0.80661943 0.19338057]
 [0.15477383 0.84522617]]


Datos con valores perdidos reemplazados por la moda:

Precisión:  0.8323633832231117
AUC:  0.8321279762509427
F1-score:  0.8182631300244225
Confusion Matrix:
  [[0.83729218 0.16270782]
 [0.17303623 0.82696377]]


Datos con atributos correlados eliminados:

Precisión:  0.826446223459798
AUC:  0.825609191849742
F1-score:  0.8105254041770446
Confusion Matrix:
  [[0.8347692  0.1652308 ]
 [0.18355082 0.81644918]]


Intentamos ajustar los hiperparámetros.

In [34]:
print("Linear Support Vector Classification")
print("C=5")
svm(less_attributes,less_target,5,None, 100000000)
print("\n")
print("C=10")
svm(less_attributes,less_target,10,None, 100000000)
print("\n")
print("C=2")
svm(less_attributes,less_target,2,None, 100000000)
print("\n")
print("C=5 y número de iteraciones menor")
svm(less_attributes,less_target,2,None, 100000000)
print("\n")
print("C=5 y clases con pesos balanceados")
svm(less_attributes,less_target,5,'balanced', 100000000)

Linear Support Vector Classification
C=5
Precisión:  0.8299617124956491
AUC:  0.8302540993848118
F1-score:  0.8236110334283498
Confusion Matrix:
  [[0.82440853 0.17559147]
 [0.16390033 0.83609967]]


C=10
Precisión:  0.8299617124956491
AUC:  0.8302540993848118
F1-score:  0.8236110334283498
Confusion Matrix:
  [[0.82440853 0.17559147]
 [0.16390033 0.83609967]]


C=2
Precisión:  0.8299547511312216
AUC:  0.8300997783971574
F1-score:  0.8233093774178657
Confusion Matrix:
  [[0.82440853 0.17559147]
 [0.16420898 0.83579102]]


C=5 y número de iteraciones menor
Precisión:  0.8299547511312216
AUC:  0.8300997783971574
F1-score:  0.8233093774178657
Confusion Matrix:
  [[0.82440853 0.17559147]
 [0.16420898 0.83579102]]


C=5 y clases con pesos balanceados
Precisión:  0.8311521058127391
AUC:  0.8314239813326136
F1-score:  0.8262881447006674
Confusion Matrix:
  [[0.81710525 0.18289475]
 [0.15425729 0.84574271]]


### Vecino más cercano kNN

In [539]:
from sklearn.neighbors import KNeighborsClassifier

def Knn(X,Y,n_neighbors,weights,p,metric='minkowski'):
    clf=KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, p=p,metric=metric)
    clasificacion(X,Y,clf)

  Empezamos determinando cual de los datos preprocesados nos da mejores resultados.

In [484]:
print("VECINO MÁS CERCANO")

print("Datos con instancias eliminadas:\n")
Knn(less_attributes,less_target,5,'uniform',2)
print("\n")

print("Datos con valores perdidos reemplazados por la moda:\n")
Knn(replaced_attributes,replaced_target,5,'uniform',2)
print("\n")

print("Datos con atributos correlados eliminados:\n")
Knn(corr_attributes,corr_target,5,'uniform',2)

VECINO MÁS CERCANO
Datos con instancias eliminadas:

Precisión:  0.7981204316045944
AUC:  0.797888272147077
F1-score:  0.7950620849451633
Confusion Matrix:
  [[0.78836964 0.21163036]
 [0.1925931  0.8074069 ]]


Datos con valores perdidos reemplazados por la moda:

Precisión:  0.7993108249216847
AUC:  0.7982967673840176
F1-score:  0.7809578062902119
Confusion Matrix:
  [[0.81197106 0.18802894]
 [0.21537753 0.78462247]]


Datos con atributos correlados eliminados:

Precisión:  0.7698085624782458
AUC:  0.7687251803454176
F1-score:  0.747574747900269
Confusion Matrix:
  [[0.78476617 0.21523383]
 [0.24731581 0.75268419]]


Comparamos entre la distancia Euclídea (por defecto) y la distancia de Hamming

In [583]:
print("VECINO MÁS CERCANO, con 5 vecinos, distancia euclídea y pesos uniformes")
Knn(less_attributes,less_target,5,'uniform', 2)
print("\n")

print("VECINO MÁS CERCANO, con 5 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,5,'uniform', 2)
print("\n")

VECINO MÁS CERCANO, con 5 vecinos, distancia euclídea y pesos uniformes
Precisión:  0.7981482770623043
AUC:  0.7998434994828142
F1-score:  0.7954530604944772
Confusion Matrix:
  [[0.78356185 0.21643815]
 [0.18387485 0.81612515]]


VECINO MÁS CERCANO, con 5 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8028750435085279
AUC:  0.8024053692594484
F1-score:  0.7982439181263353
Confusion Matrix:
  [[0.80172264 0.19827736]
 [0.1969119  0.8030881 ]]




In [584]:
print("VECINO MÁS CERCANO, con 5 vecinos, distancia de Manhattan y pesos uniformes")
Knn(less_attributes,less_target,5,'uniform', 1)
print("\n")

VECINO MÁS CERCANO, con 5 vecinos, distancia de Manhattan y pesos uniformes
Precisión:  0.7957883745214062
AUC:  0.7957903723340201
F1-score:  0.7870512179271973
Confusion Matrix:
  [[0.80276774 0.19723226]
 [0.211187   0.788813  ]]




Comparación con pesos inversamente proporcionales a la distancia

In [592]:
print("VECINO MÁS CERCANO, con 5 vecinos, distancia euclídea y pesos inversamente proporcionales a la distancia")
Knn(less_attributes,less_target,5,'distance', 2)
print("\n")

print("VECINO MÁS CERCANO, con 5 vecinos, distancia de Hamming y pesos inversamente proporcionales a la distancia")
Knn(less_attributes,less_target,5,'distance', 2,'hamming')
print("\n")

VECINO MÁS CERCANO, con 5 vecinos, distancia euclídea y pesos inversamente proporcionales a la distancia
Precisión:  0.7851305255830143
AUC:  0.784532897126519
F1-score:  0.7742808769869896
Confusion Matrix:
  [[0.80559439 0.19440561]
 [0.2365286  0.7634714 ]]


VECINO MÁS CERCANO, con 5 vecinos, distancia de Hamming y pesos inversamente proporcionales a la distancia
Precisión:  0.7946049425687434
AUC:  0.7931606933143145
F1-score:  0.7728125448801005
Confusion Matrix:
  [[0.8621373  0.1378627 ]
 [0.27581592 0.72418408]]




Probamos con diferentes números de vecinos hasta encontrar el que está próximo al óptimo.

In [572]:
#Tunning k

print("VECINO MÁS CERCANO, con 1 vecino, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,1,'uniform', 2,'hamming')
print("\n")

print("VECINO MÁS CERCANO, con 4 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,4,'uniform', 2,'hamming')
print("\n")

print("VECINO MÁS CERCANO, con 10 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,10,'uniform', 2,'hamming')
print("\n")

print("VECINO MÁS CERCANO, con 20 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,20,'uniform', 2,'hamming')
print("\n")

VECINO MÁS CERCANO, con 1 vecino, distancia de Hamming y pesos uniformes
Precisión:  0.7555516881308736
AUC:  0.7550017134079365
F1-score:  0.7378623780668422
Confusion Matrix:
  [[0.79986728 0.20013272]
 [0.28986385 0.71013615]]


VECINO MÁS CERCANO, con 4 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.7851514096762966
AUC:  0.7819217267058346
F1-score:  0.7463511213315167
Confusion Matrix:
  [[0.8994839  0.1005161 ]
 [0.33564044 0.66435956]]


VECINO MÁS CERCANO, con 10 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8134145492516532
AUC:  0.8109042249554775
F1-score:  0.7910524451696564
Confusion Matrix:
  [[0.89069953 0.10930047]
 [0.26889108 0.73110892]]


VECINO MÁS CERCANO, con 20 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8146258266620258
AUC:  0.8120754079569815
F1-score:  0.7915890507550165
Confusion Matrix:
  [[0.89287608 0.10712392]
 [0.26872526 0.73127474]]




In [570]:
print("VECINO MÁS CERCANO, con 7 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,7,'uniform', 2,'hamming')
print("\n")

print("VECINO MÁS CERCANO, con 8 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,8,'uniform', 2,'hamming')
print("\n")

print("VECINO MÁS CERCANO, con 15 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,15,'uniform', 2,'hamming')
print("\n")

print("VECINO MÁS CERCANO, con 17 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,17,'uniform', 2,'hamming')
print("\n")

VECINO MÁS CERCANO, con 7 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8229376957883746
AUC:  0.8212037488725834
F1-score:  0.8031505451230256
Confusion Matrix:
  [[0.89519431 0.10480569]
 [0.25278681 0.74721319]]


VECINO MÁS CERCANO, con 8 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8217194570135746
AUC:  0.8196990173192409
F1-score:  0.7996630552624869
Confusion Matrix:
  [[0.89735201 0.10264799]
 [0.25795397 0.74204603]]


VECINO MÁS CERCANO, con 15 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8181970066132962
AUC:  0.8154989719369954
F1-score:  0.7999403271649197
Confusion Matrix:
  [[0.87830899 0.12169101]
 [0.24731104 0.75268896]]


VECINO MÁS CERCANO, con 17 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.8098990602158022
AUC:  0.8080871065866624
F1-score:  0.7888527818999972
Confusion Matrix:
  [[0.88146513 0.11853487]
 [0.26529092 0.73470908]]




In [571]:
print("VECINO MÁS CERCANO, con 6 vecinos, distancia de Hamming y pesos uniformes")
Knn(less_attributes,less_target,6,'uniform', 2,'hamming')
print("\n")

VECINO MÁS CERCANO, con 6 vecinos, distancia de Hamming y pesos uniformes
Precisión:  0.818162199791159
AUC:  0.8174518844480131
F1-score:  0.7946248465939861
Confusion Matrix:
  [[0.90459017 0.09540983]
 [0.2696864  0.7303136 ]]




### Àrboles de decisión

In [109]:
from sklearn.tree import DecisionTreeClassifier,export_graphviz
import graphviz

def decisionTree(X,Y, criterion,depth=False,max_depth=None, min_samples_split=2,min_samples_leaf=1,plot=False):
    clf=DecisionTreeClassifier(criterion=criterion,max_depth=max_depth, min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,random_state=100)
    clasificacion(X,Y,clf)
    if depth:
        print("Max depth: ", clf.tree_.max_depth)
        print("Número de hojas: ", clf.get_n_leaves())
        print("Número de nodos: ", clf.tree_.node_count)
    if plot:
       dot_data = export_graphviz(clf, out_file=None, feature_names=X.columns, class_names=['benigno','maligno'], filled=True)
       graph = graphviz.Source(dot_data, format="png")
       graph.render('decision_tree') 
       graph
                                  

  Empezamos determinando cual de los datos preprocesados nos da mejores resultados.

In [527]:
print("DECISION TREE")

print("Datos con instancias eliminadas:\n")
decisionTree(less_attributes,less_target,'gini')
print("\n")


print("Datos con valores perdidos reemplazados por la moda:\n")
decisionTree(replaced_attributes,replaced_target,'gini')
print("\n")

DECISION TREE
Datos con instancias eliminadas:

Precisión:  0.7779951270449008
AUC:  0.7768099277338513
F1-score:  0.7586725291309332
Confusion Matrix:
  [[0.83055656 0.16944344]
 [0.2769367  0.7230633 ]]


Datos con valores perdidos reemplazados por la moda:

Precisión:  0.7839679777236339
AUC:  0.7793948178911096
F1-score:  0.7500298347871655
Confusion Matrix:
  [[0.83771273 0.16228727]
 [0.2789231  0.7210769 ]]




In [528]:
print("Datos con atributos correlados eliminados:\n")
decisionTree(corr_attributes,corr_target,'gini')

Datos con atributos correlados eliminados:

Precisión:  0.7697737556561085
AUC:  0.7688380621922528
F1-score:  0.7447295018394028
Confusion Matrix:
  [[0.79679034 0.20320966]
 [0.25911422 0.74088578]]


Determinamos ahora cual de las dos medidas de impureza (gini o entropía) nos da mejores resultados.

In [711]:
print("Datos con instancias eliminadas:\n")

print("DECISION TREE usando la medida de gini")
decisionTree(less_attributes,less_target,'gini',True)
print("\n")
print("DECISION TREE usando la entropía")
decisionTree(less_attributes,less_target,'entropy',True)

Datos con instancias eliminadas:

DECISION TREE usando la medida de gini
Precisión:  0.756728158719109
AUC:  0.7553461844825675
F1-score:  0.7351423218253031
Confusion Matrix:
  [[0.80574534 0.19425466]
 [0.29505297 0.70494703]]
Max depth:  17
Número de hojas:  193
Número de nodos:  385


DECISION TREE usando la entropía
Precisión:  0.7721266968325792
AUC:  0.7700158300497173
F1-score:  0.7492936050164565
Confusion Matrix:
  [[0.82799099 0.17200901]
 [0.28795933 0.71204067]]
Max depth:  20
Número de hojas:  188
Número de nodos:  375


Estudiamos si hay sobreajuste cambiando los valores de max_depth y buscando el óptimo.

In [703]:
print("DECISION TREE con max_depth=15")
decisionTree(less_attributes,less_target,'entropy',True, 15)
print("\n")

print("DECISION TREE con max_depth=10")
decisionTree(less_attributes,less_target,'entropy',True, 10)

print("\n")
print("DECISION TREE con max_depth=19")
decisionTree(less_attributes,less_target,'entropy', True,19)
print("\n")

print("DECISION TREE con max_depth=5")
decisionTree(less_attributes,less_target,'entropy',True, 5)
print("\n")

DECISION TREE con max_depth=15
Precisión:  0.7827149321266968
AUC:  0.7808145643329147
F1-score:  0.7619943327343985
Confusion Matrix:
  [[0.82820249 0.17179751]
 [0.26657336 0.73342664]]
Max depth:  15
Número de hojas:  168
Número de nodos:  335


DECISION TREE con max_depth=10
Precisión:  0.8028123912286809
AUC:  0.80298881340693
F1-score:  0.7889916241737553
Confusion Matrix:
  [[0.83400563 0.16599437]
 [0.22802801 0.77197199]]
Max depth:  10
Número de hojas:  100
Número de nodos:  199


DECISION TREE con max_depth=19
Precisión:  0.7756561085972851
AUC:  0.7736033391274225
F1-score:  0.754481226505251
Confusion Matrix:
  [[0.82563805 0.17436195]
 [0.27843137 0.72156863]]
Max depth:  19
Número de hojas:  187
Número de nodos:  373


DECISION TREE con max_depth=5
Precisión:  0.8157953358858336
AUC:  0.8154885233232625
F1-score:  0.8026228067873873
Confusion Matrix:
  [[0.84261577 0.15738423]
 [0.21163872 0.78836128]]
Max depth:  5
Número de hojas:  23
Número de nodos:  45




In [701]:
decisionTree(less_attributes,less_target,'entropy',True, 4)
print("\n")

decisionTree(less_attributes,less_target,'entropy',True, 6)
print("\n")

decisionTree(less_attributes,less_target,'entropy',True, 3)
print("\n")

decisionTree(less_attributes,less_target,'entropy',True, 2)
print("\n")

Precisión:  0.827615732683606
AUC:  0.8261621844016205
F1-score:  0.8110240349288036
Confusion Matrix:
  [[0.87603826 0.12396174]
 [0.22371389 0.77628611]]
Max depth:  4
Número de hojas:  13
Número de nodos:  25


Precisión:  0.8110546467107553
AUC:  0.8106310334401012
F1-score:  0.798169163162347
Confusion Matrix:
  [[0.83777118 0.16222882]
 [0.21650911 0.78349089]]
Max depth:  6
Número de hojas:  35
Número de nodos:  69


Precisión:  0.837020536025061
AUC:  0.8352894447895473
F1-score:  0.8187818975941452
Confusion Matrix:
  [[0.89923105 0.10076895]
 [0.22865216 0.77134784]]
Max depth:  3
Número de hojas:  7
Número de nodos:  13


Precisión:  0.8040236686390532
AUC:  0.8040387860262734
F1-score:  0.785271704998151
Confusion Matrix:
  [[0.85399505 0.14600495]
 [0.24591748 0.75408252]]
Max depth:  2
Número de hojas:  4
Número de nodos:  7




Intentamos buscar ahora el mejor valor para min_samples_split.

In [714]:
print("Mínimo de ejemplos en un nodo para que se pueda dividir es 10")
decisionTree(less_attributes,less_target,'entropy',True, None, 10)
print("\n")

print("Mínimo de ejemplos en un nodo para que se pueda dividir es 20")
decisionTree(less_attributes,less_target,'entropy',True, None, 20)
print("\n")

print("Mínimo de ejemplos en un nodo para que se pueda dividir es 50")
decisionTree(less_attributes,less_target,'entropy',True, None, 50)
print("\n")

print("Mínimo de ejemplos en un nodo para que se pueda dividir es 100")
decisionTree(less_attributes,less_target,'entropy',True, None, 100)
print("\n") 

print("Mínimo de ejemplos en un nodo para que se pueda dividir es 150")
decisionTree(less_attributes,less_target,'entropy',True, None, 150)
print("\n") 

Mínimo de ejemplos en un nodo para que se pueda dividir es 10
Precisión:  0.7933379742429516
AUC:  0.7934916928625583
F1-score:  0.777005503468657
Confusion Matrix:
  [[0.83575024 0.16424976]
 [0.24876685 0.75123315]]
Max depth:  16
Número de hojas:  101
Número de nodos:  201


Mínimo de ejemplos en un nodo para que se pueda dividir es 20
Precisión:  0.79686042464323
AUC:  0.7963171021430729
F1-score:  0.7849535748766355
Confusion Matrix:
  [[0.81265749 0.18734251]
 [0.22002329 0.77997671]]
Max depth:  15
Número de hojas:  61
Número de nodos:  121


Mínimo de ejemplos en un nodo para que se pueda dividir es 50
Precisión:  0.8145910198398886
AUC:  0.8138441882529148
F1-score:  0.7976758299934119
Confusion Matrix:
  [[0.86144039 0.13855961]
 [0.23375201 0.76624799]]
Max depth:  11
Número de hojas:  35
Número de nodos:  69


Mínimo de ejemplos en un nodo para que se pueda dividir es 100
Precisión:  0.8299547511312217
AUC:  0.8291954014044889
F1-score:  0.8139892051019363
Confusion Matrix:

In [713]:
print("Mínimo de ejemplos en un nodo para que se pueda dividir es 80")
decisionTree(less_attributes,less_target,'entropy',True, None, 80)
print("\n") 

print("Mínimo de ejemplos en un nodo para que se pueda dividir es 120")
decisionTree(less_attributes,less_target,'entropy',True, None, 120)
print("\n") 


print("Mínimo de ejemplos en un nodo para que se pueda dividir es 110")
decisionTree(less_attributes,less_target,'entropy',True, None, 110)
print("\n") 


print("Mínimo de ejemplos en un nodo para que se pueda dividir es 115")
decisionTree(less_attributes,less_target,'entropy',True, None, 115)
print("\n")  

Mínimo de ejemplos en un nodo para que se pueda dividir es 80
Precisión:  0.8240375913679081
AUC:  0.8245145885304878
F1-score:  0.8127540556360175
Confusion Matrix:
  [[0.84571005 0.15428995]
 [0.19668088 0.80331912]]
Max depth:  9
Número de hojas:  20
Número de nodos:  39


Mínimo de ejemplos en un nodo para que se pueda dividir es 120
Precisión:  0.8134632788026452
AUC:  0.8143344636877459
F1-score:  0.8036441762817692
Confusion Matrix:
  [[0.82750627 0.17249373]
 [0.19883734 0.80116266]]
Max depth:  7
Número de hojas:  14
Número de nodos:  27


Mínimo de ejemplos en un nodo para que se pueda dividir es 110
Precisión:  0.8311103376261748
AUC:  0.8304115895026994
F1-score:  0.8160024199572563
Confusion Matrix:
  [[0.87694447 0.12305553]
 [0.21612129 0.78387871]]
Max depth:  7
Número de hojas:  15
Número de nodos:  29


Mínimo de ejemplos en un nodo para que se pueda dividir es 115
Precisión:  0.8322868082144101
AUC:  0.8316461574039339
F1-score:  0.8174816081237509
Confusion Matrix:


Finalmente estudiamos los resultados para distintos valores de min_samples_leaf.

In [722]:
print("Mínimo de ejemplos que debe tener un nodo hoja es 30")
decisionTree(less_attributes,less_target,'entropy',True, None,2,30)
print("\n") 

print("Mínimo de ejemplos que debe tener un nodo hoja es 40")
decisionTree(less_attributes,less_target,'entropy',True, None,2,40)
print("\n") 

print("Mínimo de ejemplos que debe tener un nodo hoja es 50")
decisionTree(less_attributes,less_target,'entropy',True, None, 2,50)
print("\n") 

print("Mínimo de ejemplos que debe tener un nodo hoja es 55")
decisionTree(less_attributes,less_target,'entropy',True, None,2,55)
print("\n") 

print("Mínimo de ejemplos que debe tener un nodo hoja es 60")
decisionTree(less_attributes,less_target,'entropy',True, None,2,60)
print("\n") 

print("Mínimo de ejemplos que debe tener un nodo hoja es 100")
decisionTree(less_attributes,less_target,'entropy',True, None,2,100)
print("\n") 

Mínimo de ejemplos que debe tener un nodo hoja es 30
Precisión:  0.8417821092934215
AUC:  0.8401078480159866
F1-score:  0.8275846830500685
Confusion Matrix:
  [[0.87416173 0.12583827]
 [0.19394604 0.80605396]]
Max depth:  6
Número de hojas:  16
Número de nodos:  31


Mínimo de ejemplos que debe tener un nodo hoja es 40
Precisión:  0.8452975983292725
AUC:  0.843296425687365
F1-score:  0.833859863341672
Confusion Matrix:
  [[0.86812209 0.13187791]
 [0.18152924 0.81847076]]
Max depth:  6
Número de hojas:  11
Número de nodos:  21


Mínimo de ejemplos que debe tener un nodo hoja es 50
Precisión:  0.8452975983292725
AUC:  0.843296425687365
F1-score:  0.8342708756366086
Confusion Matrix:
  [[0.86576915 0.13423085]
 [0.1791763  0.8208237 ]]
Max depth:  5
Número de hojas:  10
Número de nodos:  19


Mínimo de ejemplos que debe tener un nodo hoja es 55
Precisión:  0.835851026801253
AUC:  0.8351712406737866
F1-score:  0.8295292116849893
Confusion Matrix:
  [[0.82968218 0.17031782]
 [0.1593397  0.8

Juntamos los tres valores óptimos y vemos los resultados

In [103]:
decisionTree(less_attributes,less_target,'entropy',True, 3,115,50)

Precisión:  0.8393804385659589
AUC:  0.8381840661368033
F1-score:  0.8309056443766405
Confusion Matrix:
  [[0.84554443 0.15445557]
 [0.1691763  0.8308237 ]]
Max depth:  3
Número de hojas:  7
Número de nodos:  13


Ahora visualizamos los árboles con los parámetros con los que hemos obtenido mejores resultados.

In [110]:
print("Mínimo de ejemplos en un nodo para que se considere una hoja es 50")
decisionTree(less_attributes,less_target,'entropy',True, None,2,50,True)

Mínimo de ejemplos en un nodo para que se considere una hoja es 50
Precisión:  0.8452975983292725
AUC:  0.843296425687365
F1-score:  0.8342708756366086
Confusion Matrix:
  [[0.86576915 0.13423085]
 [0.1791763  0.8208237 ]]
Max depth:  5
Número de hojas:  10
Número de nodos:  19


In [111]:
print("Profundidad máxima es 3")
decisionTree(less_attributes,less_target,'entropy',True, 3,2,1,True)

Profundidad máxima es 3
Precisión:  0.837020536025061
AUC:  0.8352894447895473
F1-score:  0.8187818975941452
Confusion Matrix:
  [[0.89923105 0.10076895]
 [0.22865216 0.77134784]]
Max depth:  3
Número de hojas:  7
Número de nodos:  13


In [91]:
print("Mínimo de ejemplos en un nodo para que se pueda dividir es 115")
decisionTree(less_attributes,less_target,'entropy',True, None,115,1,True)

Mínimo de ejemplos en un nodo para que se pueda dividir es 115
Precisión:  0.8322868082144101
AUC:  0.8316461574039339
F1-score:  0.8174816081237509
Confusion Matrix:
  [[0.87694447 0.12305553]
 [0.21365216 0.78634784]]
Max depth:  7
Número de hojas:  14
Número de nodos:  27


### Random Forest

In [2]:
from sklearn.ensemble import RandomForestClassifier

def RanForest(X,Y,num_trees,criterion,max_depth=None, bootstrap=True,min_samples_split=2,min_samples_leaf=1):
    clf=RandomForestClassifier(n_estimators=num_trees, criterion=criterion,max_depth=max_depth, bootstrap=bootstrap,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,random_state=100)
    clasificacion(X,Y,clf)

Determinar cuales de los datos preprocesados es mejor.

In [461]:
print("RANDOM FOREST")

print("Datos con instancias eliminadas:\n")
RanForest(less_attributes,less_target,200,'gini')
print("\n")

print("Datos con valores perdidos reemplazados por la moda:\n")
RanForest(replaced_attributes,replaced_target,200,'gini')
print("\n")

RANDOM FOREST
Datos con instancias eliminadas:

Precisión:  0.8087504350852767
AUC:  0.8086506961702474
F1-score:  0.8013035065117279
Confusion Matrix:
  [[0.81535044 0.18464956]
 [0.19804905 0.80195095]]


Datos con valores perdidos reemplazados por la moda:

Precisión:  0.8134702401670728
AUC:  0.8150029742851359
F1-score:  0.7952746093769246
Confusion Matrix:
  [[0.82865343 0.17134657]
 [0.19864748 0.80135252]]




In [487]:
print("Datos con atributos correlados eliminados:\n")
RanForest(corr_attributes,corr_target,200,'gini')

Datos con atributos correlados eliminados:

Precisión:  0.7980577793247476
AUC:  0.7985537437360607
F1-score:  0.7819330029314434
Confusion Matrix:
  [[0.80523968 0.19476032]
 [0.20813219 0.79186781]]


Intentamos determinar ahora el número de árboles óptimo

In [34]:
print("RANDOM FOREST con 200 árboles")
RanForest(less_attributes,less_target,200,'entropy',None,True,2,50)
print("\n")

print("RANDOM FOREST con 100 árboles")
RanForest(less_attributes,less_target,100,'entropy',None,True,2,50)
print("\n")

print("RANDOM FOREST con 300 árboles")
RanForest(less_attributes,less_target,300,'entropy',None,True,2,50)
print("\n")

print("RANDOM FOREST con 150 árboles")
RanForest(less_attributes,less_target,150,'entropy',None,True,2,50)
print("\n")

print("RANDOM FOREST con 50 árboles")
RanForest(less_attributes,less_target,50,'entropy',None,True,2,50)
print("\n")

print("RANDOM FOREST con 10 árboles")
RanForest(less_attributes,less_target,10,'entropy',None,True,2,50)
print("\n")

RANDOM FOREST con 200 árboles
Precisión:  0.8417542638357117
AUC:  0.8399088333471827
F1-score:  0.8269787044432089
Confusion Matrix:
  [[0.88274116 0.11725884]
 [0.20292349 0.79707651]]


RANDOM FOREST con 100 árboles
Precisión:  0.8417473024712845
AUC:  0.8400354154747562
F1-score:  0.8290809588813751
Confusion Matrix:
  [[0.87080478 0.12919522]
 [0.19073395 0.80926605]]


RANDOM FOREST con 300 árboles
Precisión:  0.8405777932474765
AUC:  0.8389441075359094
F1-score:  0.824836783089357
Confusion Matrix:
  [[0.88769151 0.11230849]
 [0.20980329 0.79019671]]


RANDOM FOREST con 150 árboles
Precisión:  0.8441141663766099
AUC:  0.8424741928243069
F1-score:  0.8301871535042986
Confusion Matrix:
  [[0.88274116 0.11725884]
 [0.19779278 0.80220722]]


RANDOM FOREST con 50 árboles
Precisión:  0.8393873999303864
AUC:  0.8378463217765194
F1-score:  0.8269920093401876
Confusion Matrix:
  [[0.8663104  0.1336896 ]
 [0.19061776 0.80938224]]


RANDOM FOREST con 10 árboles
Precisión:  0.83702053602506

In [39]:
print("RANDOM FOREST con 150 árboles sin bootstrap")
RanForest(less_attributes,less_target,150,'entropy',None,False,2,50)
print("\n")

RANDOM FOREST con 150 árboles sin bootstrap
Precisión:  0.8358440654368255
AUC:  0.8340731044914417
F1-score:  0.8219159954649733
Confusion Matrix:
  [[0.8710697  0.1289303 ]
 [0.20292349 0.79707651]]




### Redes neuronales

In [35]:
from sklearn.neural_network import MLPClassifier

def NN(X,Y,hl_sizes,activation='relu',solver='adam',alpha=0.0001,learning_rate=0.001,max_iter=200):
    clf=MLPClassifier(hidden_layer_sizes=hl_sizes, activation=activation,solver=solver,alpha=alpha, learning_rate_init=learning_rate, max_iter=max_iter,random_state=100)
    clasificacion(X,Y,clf)

Determinar cuales de los datos preprocesados es mejor.

In [531]:
print("NEURAL NETWORK")

print("Datos con instancias eliminadas:\n")
NN(less_attributes,less_target,(100),'relu','adam',0.001,1000)
print("\n")

print("Datos con valores perdidos reemplazados por la moda:\n")
NN(replaced_attributes,replaced_target,(100),'relu','adam',0.001,1000)
print("\n")

NEURAL NETWORK
Datos con instancias eliminadas:

Precisión:  0.8016150365471633
AUC:  0.8023870371400657
F1-score:  0.803364631478301
Confusion Matrix:
  [[0.76453855 0.23546145]
 [0.15976448 0.84023552]]


Datos con valores perdidos reemplazados por la moda:

Precisión:  0.8063487643578142
AUC:  0.8082588929874106
F1-score:  0.7926877734449324
Confusion Matrix:
  [[0.79285925 0.20714075]
 [0.17634147 0.82365853]]




In [532]:
print("Datos con atributos correlados eliminados:\n")
NN(corr_attributes,corr_target,(100),'relu','adam',0.001,1000)

Datos con atributos correlados eliminados:

Precisión:  0.7933727810650888
AUC:  0.7940723507021803
F1-score:  0.7776191376369422
Confusion Matrix:
  [[0.79676006 0.20323994]
 [0.20861536 0.79138464]]


Pasamos ahora a configurar en lo posible algunos de los hiperparámetros.

In [48]:
from sklearn.model_selection import GridSearchCV

mlp=MLPClassifier()
parameter_space = {
    'hidden_layer_sizes': [(100,),(100,50),(200,100,50),(10,20,10)],
    'solver': ['lbfgs'],
    'alpha': [0.001, 0.05,0.01],
    'max_iter':[1000,10000],
}

clf=GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(less_attributes,less_target)

GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.05, 0.01],
                         'hidden_layer_sizes': [(100,), (100, 50),
                                                (200, 100, 50), (10, 20, 10)],
                         'max_iter': [1000, 10000], 'solver': ['lbfgs']})

In [49]:
print('Mejores parámetros: ', clf.best_params_)

Mejores parámetros:  {'alpha': 0.001, 'hidden_layer_sizes': (10, 20, 10), 'max_iter': 10000, 'solver': 'lbfgs'}


In [53]:
print("NEURAL NETWORK con los mejores parámetros obtenidos")
NN(less_attributes,less_target,(10,20,10),'relu','lbfgs',0.001,0.0001,10000)

NEURAL NETWORK con los mejores parámetros obtenidos
Precisión:  0.8429307344239471
AUC:  0.8424255921351396
F1-score:  0.8328184297541081
Confusion Matrix:
  [[0.86097193 0.13902807]
 [0.17612074 0.82387926]]


In [72]:
print("NEURAL NETWORK con max_iter=100000 y mejores parámetros")
NN(less_attributes,less_target,(10,20,10),'relu','lbfgs',0.001,0.0001,100000)

NEURAL NETWORK con max_iter=100000 y mejores parámetros
Precisión:  0.8429307344239471
AUC:  0.8424255921351396
F1-score:  0.8328184297541081
Confusion Matrix:
  [[0.86097193 0.13902807]
 [0.17612074 0.82387926]]


In [55]:
mlp=MLPClassifier()
parameter_space = {
    'hidden_layer_sizes': [(25,50,25),(10,20,10),(5,10,5),(10,20)],
    'solver': ['lbfgs'],
    'alpha': [0.001, 0.05,0.01],
    'max_iter':[10000],
}

clf=GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(less_attributes,less_target)

GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.05, 0.01],
                         'hidden_layer_sizes': [(25, 50, 25), (10, 20, 10),
                                                (5, 10, 5), (10, 20)],
                         'max_iter': [10000], 'solver': ['lbfgs']})

In [56]:
print('Mejores parámetros: ', clf.best_params_)

Mejores parámetros:  {'alpha': 0.05, 'hidden_layer_sizes': (25, 50, 25), 'max_iter': 10000, 'solver': 'lbfgs'}


In [74]:
print("NEURAL NETWORK con mejores parámetros")
NN(less_attributes,less_target,(25,50,25),'relu','lbfgs',0.05,0.001,10000)

NEURAL NETWORK con mejores parámetros
Precisión:  0.8429237730595197
AUC:  0.8425369669307745
F1-score:  0.8334203868879484
Confusion Matrix:
  [[0.85835406 0.14164594]
 [0.17328013 0.82671987]]


In [83]:
mlp=MLPClassifier()
parameter_space = {
    'hidden_layer_sizes': [(25,50,25),(50,100,50),(100,150,100),(100,50,25),(25,25,25)],
    'solver': ['lbfgs'],
    'alpha': [0.0001,0.001, 0.05,0.1,0.25,0.08],
    'max_iter':[10000],
}

clf=GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(less_attributes,less_target)

GridSearchCV(cv=5, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.001, 0.05, 0.1, 0.25, 0.08],
                         'hidden_layer_sizes': [(25, 50, 25), (50, 100, 50),
                                                (100, 150, 100), (100, 50, 25),
                                                (25, 25, 25)],
                         'max_iter': [10000], 'solver': ['lbfgs']})

In [84]:
print('Mejores parámetros: ', clf.best_params_)

Mejores parámetros:  {'alpha': 0.25, 'hidden_layer_sizes': (25, 25, 25), 'max_iter': 10000, 'solver': 'lbfgs'}


In [85]:
print("NEURAL NETWORK con mejores parámetros")
NN(less_attributes,less_target,(25,25,25),'relu','lbfgs',0.25,0.0001,10000)

NEURAL NETWORK con mejores parámetros
Precisión:  0.8346954403063002
AUC:  0.8345448343435866
F1-score:  0.8239382898700367
Confusion Matrix:
  [[0.85859853 0.14140147]
 [0.18950886 0.81049114]]


Otras configuraciones.

In [70]:
print("NEURAL NETWORK")
NN(less_attributes,less_target,(100,50,25),'relu','lbfgs',0.08,0.001,10000)

NEURAL NETWORK
Precisión:  0.8429098503306648
AUC:  0.8416928818272679
F1-score:  0.8311054428699279
Confusion Matrix:
  [[0.86684595 0.13315405]
 [0.18346018 0.81653982]]


In [76]:
print("NEURAL NETWORK")
NN(less_attributes,less_target,(10,20,10),'relu','lbfgs',0.05,0.001,10000)

NEURAL NETWORK
Precisión:  0.835864949530108
AUC:  0.8358058678916092
F1-score:  0.8252829003886208
Confusion Matrix:
  [[0.85883048 0.14116952]
 [0.18721875 0.81278125]]


In [77]:
print("NEURAL NETWORK")
NN(less_attributes,less_target,(100,50,25),'relu','lbfgs',0.05,0.001,10000)

NEURAL NETWORK
Precisión:  0.8358371040723982
AUC:  0.8343120992354989
F1-score:  0.821845353228332
Confusion Matrix:
  [[0.86948997 0.13051003]
 [0.20086578 0.79913422]]
